In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import random
import time

In [ ]:
URL = 'https://www.flipkart.com/search?q=smart+lock&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=6'
API_KEY = '53fde5daee9e3fd8b128656faaf77e49'

# ScraperAPI setup
api_url = f'http://api.scraperapi.com?api_key={API_KEY}&url={URL}'

In [ ]:
# HTTP Request
webpage = requests.get(api_url)
# Soup Object containiang all data
soup = BeautifulSoup(webpage.content, "html.parser")

In [ ]:
links = soup.find_all("a", attrs={'class':'wjcEIp'})

In [ ]:
print(len(links))

34


In [ ]:
# Initialize the list to store product data
product_data = []

for i in range(len(links)):
    try:
        link = links[i].get('href')
        product_link_full = "https://www.flipkart.com" + link
        product_url = f'http://api.scraperapi.com?api_key={API_KEY}&url={product_link_full}'

        # Request product page
        product_webpage = requests.get(product_url)
        product_soup = BeautifulSoup(product_webpage.content, "html.parser")

        # Extract details with error handling
        try:
            title = product_soup.find("span", attrs={"class": 'VU-ZEz'}).text.strip()
        except AttributeError:
            title = "0"

        try:
            brand = product_soup.find("td", attrs={"class": 'Izz52n col col-9-12'}).text.strip()
        except AttributeError:
            brand = "0"

        try:
            price = product_soup.find("div", attrs={"class": 'Nx9bqj CxhGGd'}).text.strip()
        except AttributeError:
            price = "0"

        try:
            rating = product_soup.find("div", attrs={"class": 'XQDdHH'}).text.strip()
        except AttributeError:
            rating = "0"

        # Handle rating count and review count
        try:
            rating_count_div = product_soup.find("div", attrs={"class": 'row j-aW8Z'})
            rating_count = rating_count_div.find("span").text.strip() if rating_count_div else "0"

            # Find the sibling for review count if rating_count_div exists
            if rating_count_div:
                review_count_div = rating_count_div.find_next_sibling("div", attrs={"class": 'row j-aW8Z'})
                review_count = review_count_div.find("span").text.strip() if review_count_div else "0"
            else:
                review_count = "0"
        except AttributeError:
            rating_count = "0"
            review_count = "0"

        # Store product data
        product_data.append({
            "Title": title,
            "Brand": brand,
            "Price": price,
            "Rating": rating,
            "Rating Count": rating_count,
            "Review Count": review_count,
            "URL": product_link_full
        })

    except Exception as e:
        print(f"Error processing product at index {i}: {e}")

# Save data to a CSV file
df = pd.DataFrame(product_data)
df.to_csv("/content/csvfiles/product_data6.csv", index=False)
print("Data saved to product_data.csv")


Data saved to product_data.csv
